### Contents

- [EDA](#EDA)

### Header

In [1]:
# user configuration

In [2]:
# import libraries

# maths
import numpy as np
import pandas as pd

# visual
#from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pydotplus

# modelling
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix,accuracy_score,r2_score,mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.utils import resample, shuffle
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.externals.six import StringIO
from imblearn.over_sampling import SMOTE

# Others
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
# file paths

folder = 'fitrec/'

input_path = '../data/1_input/' + folder
clean_path = '../data/2_clean/' + folder
preprocess_path = '../data/3_preprocess/' + folder
output_path = '../data/4_output/' + folder

### Import Data

In [4]:
# import data

file = 'endomondoHR_proper_summary.csv'
in_path = clean_path + file

df = pd.read_csv(in_path)

### Functions

### Pre-process Data

In [5]:
df.columns

Index(['id', 'userId', 'gender', 'sport', 'time_start', 'time_end', 'time_dur',
       'lat_start', 'lat_end', 'lon_start', 'lon_end', 'alt_avg', 'alt_min',
       'alt_q1', 'alt_q3', 'alt_max', 'hr_avg', 'hr_min', 'hr_q1', 'hr_q3',
       'hr_max', 'hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak', 'spd_avg',
       'spd_min', 'spd_q1', 'spd_q3', 'spd_max'],
      dtype='object')

In [6]:
# remove rows with abnormal heartrate

print(len(df))

cond_1 = df['hr_avg'] > 60
cond_2 = df['hr_min'] > 0
cond_3 = df['hr_q1'] > 0
cond_4 = df['hr_q3'] > 0
cond_5 = df['hr_max'] > 0

df = df[cond_1 & cond_2 & cond_3 & cond_4 & cond_5]

print(len(df))

167783
167219


In [7]:
# drop rows if speed is nan

# print(len(df))
# df.dropna(subset=['spd_avg'],inplace=True)
# print(len(df))

In [8]:
df['sport'].value_counts()

bike                       71652
run                        70468
mountain bike              10710
bike (transport)            7658
indoor cycling              1725
walk                        1246
orienteering                 867
cross-country skiing         788
core stability training      443
fitness walking              292
skate                        259
roller skiing                238
hiking                       237
kayaking                      92
circuit training              89
weight training               74
rowing                        71
gymnastics                    66
soccer                        51
downhill skiing               42
treadmill running             28
snowshoeing                   16
swimming                      14
golf                          12
badminton                     10
elliptical                    10
horseback riding              10
basketball                     8
tennis                         8
aerobics                       7
climbing  

In [9]:
# merge similar sports

df['sport'].replace('treadmill running','run',inplace=True)
df['sport'].replace(['treadmill walking','fitness walking'],'walk',inplace=True)
df['sport'].replace(['bike','mountain bike','bike (transport)','indoor cycling'],'cycle',inplace=True)
df['sport'].replace(['windsurfing','kite surfing'],'surfing',inplace=True)
df['sport'].replace(['cross-country skiing','downhill skiing','roller skiing'],'skiing',inplace=True)
df['sport'].replace('pilates','yoga',inplace=True)

In [10]:
df['sport'].value_counts()

cycle                      91745
run                        70496
walk                        1539
skiing                      1068
orienteering                 867
core stability training      443
skate                        259
hiking                       237
kayaking                      92
circuit training              89
weight training               74
rowing                        71
gymnastics                    66
soccer                        51
snowshoeing                   16
swimming                      14
golf                          12
elliptical                    10
badminton                     10
horseback riding              10
tennis                         8
basketball                     8
aerobics                       7
climbing                       5
yoga                           4
table tennis                   4
snowboarding                   3
stair climing                  3
rugby                          3
surfing                        2
sailing   

In [11]:
# select columns for feature selection

# cols = ['sport', 'time_dur', 'alt_avg', 'alt_min', 'alt_q1', 'alt_q3','alt_max', 'hr_avg', 'hr_min', 'hr_q1', 'hr_q3', 'hr_max']

#cols = ['sport','hr_avg', 'hr_min', 'hr_q1', 'hr_q3', 'hr_max']
#cols = ['sport','hr_avg', 'hr_min', 'hr_q1', 'hr_q3', 'hr_max','spd_avg', 'spd_min', 'spd_q1', 'spd_q3', 'spd_max']

cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak']
#cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak','spd_avg', 'spd_min', 'spd_q1', 'spd_q3', 'spd_max']

df_model = df[cols]

In [12]:
# select only sports with minimal rows

count = df_model['sport'].value_counts()

#count_cond = count[count > 1].index
#count_cond = count[count >= 5].index
#count_cond = count[count >= 10].index
#count_cond = count[count >= 50].index
#count_cond = count[count >= 100].index
count_cond = count[count >= 800].index

count_mask = df_model['sport'].isin(count_cond)
df_model = df_model[count_mask]

In [13]:
print(df_model.shape)
df_model.head()

(165715, 5)


,sport,hr_outof,hr_fatburn,hr_cardio,hr_peak
0,cycle,0.0,0.013333,0.464444,0.522222
1,cycle,0.0,0.020000,0.591111,0.388889
2,cycle,0.0,0.057778,0.782222,0.160000
3,cycle,0.0,0.037778,0.637778,0.324444
4,cycle,0.0,0.011111,0.080000,0.908889


In [14]:
df_model['sport'].value_counts()

cycle           91745
run             70496
walk             1539
skiing           1068
orienteering      867
Name: sport, dtype: int64

### Plot df_model

In [15]:
# order = df_model.groupby('sport')['hr_max'].median().sort_values(ascending=False).index

# plt.figure(figsize=(20,15))
# #plt.xlim(0,300)

# sns.boxplot(data=df_model,x='hr_max',y='sport',order=order);

In [16]:
# order = df_model.groupby('sport')['hr_avg'].median().sort_values(ascending=False).index

# plt.figure(figsize=(20,15))
# #plt.xlim(0,300)

# sns.boxplot(data=df_model,x='hr_avg',y='sport',order=order);

In [17]:
# order = df_model.groupby('sport')['hr_min'].median().sort_values(ascending=False).index

# plt.figure(figsize=(20,15))
# #plt.xlim(0,300)

# sns.boxplot(data=df_model,x='hr_min',y='sport',order=order);

### Create Features and Target

In [18]:
target = 'sport'
features = [x for x in df_model.columns if x != target]

X = df_model[features].values
y = df_model[target].values

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,stratify=y,random_state=3050)

In [19]:
ss = StandardScaler()
ss.fit(X_train)

X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

### Handle Imbalance Data

In [20]:
# check before smote

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(132572, 4)
(33143, 4)
(132572,)
(33143,)


In [21]:
# check before smote

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 73396, 1: 694, 2: 56397, 3: 854, 4: 1231}

In [22]:
sm = SMOTE(random_state=3050)
X_train, y_train = sm.fit_sample(X_train, y_train.ravel())

In [23]:
# check after smote

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(366980, 4)
(33143, 4)
(366980,)
(33143,)


In [24]:
# check after smote

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 73396, 1: 73396, 2: 73396, 3: 73396, 4: 73396}

### Logistic Regresion Model

In [25]:
# init model

logreg = LogisticRegression()

In [26]:
# perform cross validation

score = cross_val_score(logreg,X,y,cv=5)
print('score:',score.mean(),score)

# score: 0.6545580125487176 [0.6302521  0.66461016 0.66203607 0.65096449 0.66492724]

score: 0.7094587790099556 [0.71853975 0.70869539 0.71939776 0.69398347 0.70667753]


In [27]:
# fit model

logreg.fit(X_train, y_train)

# score model
print("train r2:",logreg.score(X_train, y_train))
print("test r2:",logreg.score(X_test, y_test))

# train r2: 0.6561339556433755
# test r2: 0.6566573926868045

train r2: 0.4138563409450106
test r2: 0.3833690372024259


In [28]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = logreg.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = logreg.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_) 

rows:actual columns:predicted


,cycle,orienteering,run,skiing,walk
cycle,6145,2935,4504,498,4267
orienteering,37,87,37,2,10
run,1524,5462,6238,159,716
skiing,50,44,59,3,58
walk,49,0,21,5,233


### KNN Model

In [29]:
# init model

knn = KNeighborsClassifier()

#n_neighbors=2
#n_neighbors=3
#n_neighbors=4
#n_neighbors=5

In [30]:
# perform cross validation

score = cross_val_score(knn,X,y,cv=5)
print('score:',score.mean(),score)

score: 0.7302357366263206 [0.72970282 0.71874246 0.74139336 0.72943697 0.73190308]


In [31]:
# fit model

knn.fit(X_train, y_train)

# score model
print("train r2:",knn.score(X_train, y_train))
print("test r2:",knn.score(X_test, y_test))

train r2: 0.8478581939070249
test r2: 0.5698940952840721


In [32]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = knn.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_)

rows:actual columns:predicted


,cycle,orienteering,run,skiing,walk
cycle,10291,1213,3457,1977,1411
orienteering,45,19,87,14,8
run,3269,1120,8426,841,443
skiing,101,22,50,27,14
walk,125,11,24,23,125


### DTC Model

In [33]:
# init model

dtc = DecisionTreeClassifier(max_depth=10,random_state=3050)

In [34]:
# perform cross validation

score = cross_val_score(dtc,X,y,cv=5)
print(score.mean(),score)

0.7578794566804519 [0.75981294 0.75181028 0.76927255 0.75140305 0.75709846]


In [35]:
# fit model
dtc = dtc.fit(X_train,y_train)

# score model
print("train r2:",dtc.score(X_train, y_train))
print("test r2:",dtc.score(X_test, y_test))

train r2: 0.5955092920595128
test r2: 0.5796095706484024


In [36]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = dtc.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = dtc.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_)

rows:actual columns:predicted


,cycle,orienteering,run,skiing,walk
cycle,10514,1138,2719,2370,1608
orienteering,54,41,62,11,5
run,2579,1813,8456,837,414
skiing,94,15,54,32,19
walk,90,3,18,30,167


In [37]:
# # initialize the output file object
# dot_data = StringIO() 

# # my fit DecisionTreeRegressor object here is: dtr1
# # for feature_names i put the columns of my Xr matrix
# export_graphviz(dtc, 
#                 out_file=dot_data,  
#                 filled=True, 
#                 rounded=True,
#                 special_characters=True,
#                 feature_names=df_model[features].columns
#                )  

# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

### RFC Model

In [38]:
# init model

rfc = RandomForestClassifier(n_estimators=10,max_depth=10,n_jobs=2, random_state=3050)

In [39]:
# perform cross validation

score = cross_val_score(rfc,X,y,cv=5)
print(score.mean(),score)

0.7606553228115269 [0.76117061 0.7553705  0.77277253 0.75339448 0.76056848]


In [40]:
# fit model
rfc = rfc.fit(X_train,y_train)

# score model
print("train r2:",rfc.score(X_train, y_train))
print("test r2:",rfc.score(X_test, y_test))

train r2: 0.6472641560848003
test r2: 0.6000965513079685


In [41]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = rfc.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = rfc.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_)

rows:actual columns:predicted


,cycle,orienteering,run,skiing,walk
cycle,10860,737,3153,2005,1594
orienteering,57,43,61,9,3
run,2513,1753,8774,669,390
skiing,92,15,51,39,17
walk,94,2,18,21,173


### GridSearch Model

In [42]:
# init models

estimators = {
    'lr': LogisticRegression(),
    'knn': KNeighborsClassifier(n_neighbors=5),
    'dtc': DecisionTreeClassifier(),
    'rfc': RandomForestClassifier(),
    'abc': AdaBoostClassifier(),
    'gbc': GradientBoostingClassifier()
}.items()

In [43]:
# init model parameters

params = {
    'lr': {

    },
    'knn': {
        'knn__n_neighbors': [3,5,7,9]
    },

    'dtc': {
        'dtc__max_features': ['auto', 'log2', None],
        'dtc__max_depth': [None, 5, 10, 15],
        'dtc__min_samples_split': np.linspace(0.1, 0.5, 5)
    },
    'rfc': {
        'rfc__n_estimators': [10, 20, 50, 100],
        'rfc__max_depth': [None, 5, 10, 15],
        'rfc__max_features': ['auto', 'log2', None],
        'rfc__min_samples_split': np.linspace(0.1, 0.5, 5)
    },
    'abc': {

    }, 
    'gbc': {

    }
}

In [44]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,stratify=y,random_state=3050)

In [50]:
models = []
parameters = []
best_score = []
roc_auc = []

for k,v in estimators:
    
    pipe = Pipeline([
        ('sc', StandardScaler()),
        (k,v)])
    
    gridsearch = GridSearchCV(
        estimator=pipe,
        param_grid=params[k],
        verbose=1,
        cv= 5,
        n_jobs=3,
        return_train_score= True
        #scoring = 'roc_auc'
    )

    gridsearch.fit(X_train, y_train)
    
    model = gridsearch.best_estimator_
    cv_score = gridsearch.cv_results_
    best_params = gridsearch.best_params_

    # predict y
    #y_pred = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    
    # print results
    print("Model: ", k)
    print("Best parameters:", best_params)
    print("Best score:", gridsearch.best_score_)
    #print("AUC/ROC test:", roc_auc_score(y_test,y_pred))
    display(pd.DataFrame(cv_score, columns = cv_score.keys()))
    
    
    # append info to list
    models.append(k)
    best_score.append(gridsearch.best_score_)
    parameters.append(best_params)
    #roc_auc.append(roc_auc_score(y_test,y_pred))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    7.7s finished


Model:  lr
Best parameters: {}
Best score: 0.4137664177884353


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,2.811836,0.183775,0.018764,0.00624,{},0.411649,0.413829,0.416037,0.415955,0.411363,0.413766,0.00201,1,0.414378,0.413972,0.412954,0.413052,0.414425,0.413756,0.000636


Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,30.774358,4.169706,1.820643,0.119704,{},0.498569,0.499857,0.50983,0.510266,0.505947,0.504894,0.004893,1,0.504053,0.503697,0.503016,0.507264,0.505172,0.504641,0.001486


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed: 13.3min finished


Model:  gbc
Best parameters: {}
Best score: 0.6355359965120715


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,403.583474,47.658221,1.121612,0.19546,{},0.630858,0.639294,0.645058,0.639281,0.62319,0.635536,0.007657,1,0.636889,0.637965,0.637972,0.636511,0.641514,0.63817,0.00177
